<a href="https://colab.research.google.com/github/JyotiChowrasia05/PySpark_Steephan/blob/main/PySpark_Steephan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install PySpark

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, trim, length,to_timestamp,to_date, coalesce,when, lit, avg

In [ ]:
# Creating a Spark Session
spark = SparkSession.builder \
    .appName("MeteoriteApp") \
    .getOrCreate()

print("Spark Session Created Successfully!")

In [ ]:
# Reading the CSV file
csv_file_path = "/content/meteorite_landings_raw.csv"

df = spark.read.csv(
    csv_file_path,
    header=True,
    inferSchema=True
)

print(f"\nDataFrame created from '{csv_file_path}'")

In [ ]:
type(df)

In [ ]:
df.show()

In [ ]:
df.describe().show()

In [ ]:
df.printSchema()

In [ ]:
df_checked = df.withColumn(
    "has_lead_trail_ws",
    (length(col("name")) != length(trim(col("name"))))
    .alias("has_lead_trail_ws")
)

print("\nDataFrame with a flag for leading/trailing whitespace:")
df_checked.show()

In [ ]:
# Triming the whitr space from name column
df = df.withColumn("name", trim(col("name")))
print("\nDataFrame with 'name' column trimmed:")
df.show()

In [ ]:
type(df['year'])

In [ ]:
# convert Year column into a date type
date_formats = [
    "dd/MM/yyyy hh:mm:ss a",
    "M/d/yyyy H:mm"
]

parsed_date_exp = [to_date(col("year"), fmt) for fmt in date_formats]

# Use coalesce to pick the first non-null parsed date
df_converted = df.withColumn(
    "converted_date",
    coalesce(*parsed_date_exp) # The '*' unpacks the list into individual arguments
)

print("\nDataFrame with converted date column (handling multiple formats):")
df_converted.show(truncate=False)
df_converted.printSchema()


In [ ]:
# Overwrite the geolocation column to null values where the data is given as (0.000000, 0.000000)
zero_coords_string = "(0.000000, 0.000000)"

df_geo = df.withColumn(
    "geolocation",
    when(
        (col("geolocation") == zero_coords_string),
         lit(None)
    ).otherwise(col("geolocation"))
)

print("\nDataFrame after nullifying specific geolocation strings:")
df_geo.show(truncate=False)
df_geo.printSchema()

In [ ]:
# Average mass of Meteors which fell
df_fell = df.filter(
    (col("fall") == "Fell") &
    col("mass (g)").isNotNull() # Exclude null mass values from average
)

print("\n The records (where column 'fall' has value 'Fell' and column mass is not null):")
print('\n Total records:', df_fell.count())
df_fell.show()

avg_mass = df_fell.agg(avg("mass (g)").alias("average_mass_fell"))

print("\nAverage mass where 'fall' is 'Fell':")
avg_mass.show()

In [ ]:
# Stop the SparkSession
# spark.stop()
# print("\nSparkSession stopped.")